# Web Mining and Applied NLP (44-620)

## Final Project: Article Summarizer

### Student Name: Arnold Atchoe.
### GitHub Repository: https://github.com/kwameape123/module7-final_project

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

You should bring in code from previous assignments to help you answer the questions below.

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

### Project Description
This project involves identifying common themes or entities across the news websites listed below and then analysing an article on the most common entities.
1.	https://www.cnn.com/world
2.	https://www.bbc.com/news
3.	https://www.foxnews.com/world


### Project Libraries
Libraries will be install in virtual environment using requirements.txt and pip.

1. beautifulsoup4  # parsing webpages (HTML documents)
2. html5lib              # parsing webpages (HTML documents)  - especially if poorly formatted
3. ipykernel    # for Jupyter notebooks
4. jupyterlab   # for Jupyter notebooks
5. matplotlib           # customizing visualizations
6. requests     # make HTTP requests (a very popular Python package)
7. spacy                   # for NLP 
8. spacytextblob     # for NLP - combines spaCy and TextBlob (simpler interface)

In [9]:
# import needed libraries
from bs4 import BeautifulSoup
import requests
import html5lib
import ipykernel
import jupyterlab
import matplotlib.pyplot as plt
import spacy
import spacytextblob



In [ ]:
%pip list

Package                   Version
------------------------- --------------
annotated-types           0.7.0
anyio                     4.6.0
argon2-cffi               23.1.0
Note: you may need to restart the kernel to use updated packages.argon2-cffi-bindings      21.2.0

arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4


attrs                     24.2.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.1.0
blis                      1.3.0
build                     1.2.2.post1
catalogue                 2.0.10
certifi                   2024.8.30
cffi                      1.17.1
charset-normalizer        3.3.2
click                     8.2.1
cloudpathlib              0.21.1
colorama                  0.4.6
comm                      0.2.2
confection                0.1.5
contourpy                 1.3.2
cycler                    0.12.1
cymem                     2.0.11
debugpy                   1.8.6
decorator                 5.1.1
defusedxml                0.7.1
et-xmlfile                1.1.0
executing                 2.1.0
fastjsonschema            2.20.0
fonttools                 4.58.1
fqdn                      1.5.1
greenlet                  3.2.2
h11                       0.14.0
html5lib                  1.1
httpcore                  1.0.6
httpx                

### FETCH CONTENTS FROM NEWS WEBSITE TO FILES

In [3]:
# fetch HTML content from multiple news websites
response_1 = requests.get("https://www.cnn.com/world")
response_2 = requests.get("https://www.bbc.com/news/world")
response_3 = requests.get("https://www.foxnews.com/world")

# check if the requests were successful
if response_1.status_code == 200 and response_2.status_code == 200 and response_3.status_code == 200:
    print("Successfully fetched content from all news websites.")
else:print("Failed to fetch content from one or more news websites.")

# check the content type
print("Content type of CNN:", response_1.headers['Content-Type'])
print("Content type of BBC:", response_2.headers['Content-Type'])
print("Content type of Fox News:", response_3.headers['Content-Type'])

# view the first 500 characters of the HTML content
print("First 500 characters of CNN content:", response_1.text[:500])
print("First 500 characters of BBC content:", response_2.text[:500])
print("First 500 characters of Fox News content:", response_3.text[:500])

# save the HTML content to pickle files to avoid re-fetching
import pickle
with open('cnn_content.pkl', 'wb') as file:
    pickle.dump(response_1.text, file)
with open('bbc_content.pkl', 'wb') as file:
    pickle.dump(response_2.text, file)
with open('foxnews_content.pkl', 'wb') as file:
    pickle.dump(response_3.text, file)



Successfully fetched content from all news websites.
Content type of CNN: text/html; charset=utf-8
Content type of BBC: text/html; charset=utf-8
Content type of Fox News: text/html; charset=utf-8
First 500 characters of CNN content:   <!DOCTYPE html>
<html lang="en" data-uri="cms.cnn.com/_pages/cl9wqykfb000026p95p1gfx93@published" data-layout-uri="cms.cnn.com/_layouts/layout-no-rail/instances/world-v1@published" >
  <head>
<link rel="dns-prefetch" href="//tpc.googlesyndication.com">

<link rel="preconnect" href="//tpc.googlesyndication.com">

<link rel="dns-prefetch" href="//pagead2.googlesyndication.com">

<link rel="preconnect" href="//pagead2.googlesyndication.com">

<link rel="dns-prefetch" href="//www.googletagservices
First 500 characters of BBC content: <!DOCTYPE html><html lang="en-GB"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><title>World | Latest News &amp; Updates | BBC News</title><meta name="page.section" content="News"/><meta name="p

### READ AND PARSE CONTENT OF PICKLE FILES

In [7]:
# read contents from the pickle files
with open('cnn_content.pkl', 'rb') as file:
    cnn_content = pickle.load(file)
with open('bbc_content.pkl', 'rb') as file:
    bbc_content = pickle.load(file)
with open('foxnews_content.pkl', 'rb') as file:
    foxnews_content = pickle.load(file)

# parse the HTML content using BeautifulSoup
cnn_soup = BeautifulSoup(cnn_content, 'html5lib')
bbc_soup = BeautifulSoup(bbc_content, 'html5lib')
foxnews_soup = BeautifulSoup(foxnews_content, 'html5lib')

# find headlines 
cnn_headlines = cnn_soup.find_all(class_='container__headline-text')
bbc_headlines = bbc_soup.find_all(attrs={'data-testid':'card-headline'})
foxnews_headlines = foxnews_soup.find_all(class_='title')
# write the headlines to a text file
with open('cnn_headlines.txt', 'w') as file:
    for headline in cnn_headlines:
        file.write(headline.get_text() + '\n')
with open('bbc_headlines.txt', 'w') as file:
    for headline in bbc_headlines:
        file.write(headline.get_text() + '\n')
with open('foxnews_headlines.txt', 'w') as file:
    for headline in foxnews_headlines:
        file.write(headline.get_text() + '\n')



### DETERMINE THE MOST COMMON HEADLINE

In [18]:
# load the English NLP model
nlp = spacy.load("en_core_web_sm")
# extract common headlines among the three news websites from text files
with open('cnn_headlines.txt', 'r') as file:
    cnn_headlines = file.readlines()
with open('bbc_headlines.txt', 'r') as file:
    bbc_headlines = file.readlines()
with open('foxnews_headlines.txt', 'r') as file:
    foxnews_headlines = file.readlines()
# combine all headlines into a single list
all_headlines = cnn_headlines + bbc_headlines + foxnews_headlines
# create a dictionary to count occurrences of each headline
headline_counts = {}
for headline in all_headlines:
    headline = headline.strip()
    if headline in headline_counts:
        headline_counts[headline] += 1
    else:
        headline_counts[headline] = 1
    
# print top ten headlines and find the most common headline
most_common_headline = max(headline_counts, key=headline_counts.get)
print("top ten headline_counts:")
top_ten_headlines = sorted(headline_counts.items(), key=lambda x: x[1], reverse=True)[:10]
for headline, count in top_ten_headlines:
    print(f"{headline}: {count} occurrences")
print()
print("Most common headline:", most_common_headline)

top ten headline_counts:
Russia drops missile deployment moratorium amid tensions with Trump admin: 3 occurrences
UN official says Israel expanding Gaza operations would risk 'catastrophic consequences': 2 occurrences
Despite Trump's peace calls, Russian attacks on Ukraine double since inauguration: 2 occurrences
Texas governor seeks court order to fire top Democrat who fled in row over voting map: 2 occurrences
RFK Jr cancels $500m in funding for mRNA vaccines that counter viruses like Covid: 2 occurrences
Hiroshima marks 80 years since atomic bombing: 2 occurrences
Great Barrier Reef suffers worst coral decline on record: 2 occurrences
Clintons subpoenaed to testify in congressional Epstein investigation: 2 occurrences
Brother of Israeli hostage urges UN to act after video shows Hamas starving and torturing captives: 2 occurrences
Johnson dines with Netanyahu in landmark visit, highest US official to visit disputed West Bank: 2 occurrences

Most common headline: Russia drops missile 

1. Find on the internet an article or blog post about a topic that interests you and you are able to get the text for using the technologies we have applied in the course.  Get the html for the article and store it in a file (which you must submit with your project)

2. Read in your article's html source from the file you created in question 1 and do sentiment analysis on the article/post's text (use `.get_text()`).  Print the polarity score with an appropriate label.  Additionally print the number of sentences in the original article (with an appropriate label)

3. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent tokens (converted to lower case).  Print the common tokens with an appropriate label.  Additionally, print the tokens their frequencies (with appropriate labels)

4. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent lemmas (converted to lower case).  Print the common lemmas with an appropriate label.  Additionally, print the lemmas with their frequencies (with appropriate labels).

5. Make a list containing the scores (using tokens) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores. From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

6. Make a list containing the scores (using lemmas) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores.  From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

7. Using the histograms from questions 5 and 6, decide a "cutoff" score for tokens and lemmas such that fewer than half the sentences would have a score greater than the cutoff score.  Record the scores in this Markdown cell

* Cutoff Score (tokens): 
* Cutoff Score (lemmas):

Feel free to change these scores as you generate your summaries.  Ideally, we're shooting for at least 6 sentences for our summary, but don't want more than 10 (these numbers are rough estimates; they depend on the length of your article).

8. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on tokens) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

9. Print the polarity score of your summary you generated with the token scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

10. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on lemmas) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

11. Print the polarity score of your summary you generated with the lemma scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

12.  Compare your polarity scores of your summaries to the polarity scores of the initial article.  Is there a difference?  Why do you think that may or may not be?.  Answer in this Markdown cell.  

13. Based on your reading of the original article, which summary do you think is better (if there's a difference).  Why do you think this might be?